In [86]:
import boto3
from dotenv import load_dotenv
%run "redshift_tables.ipynb"
import os
load_dotenv()

True

In [87]:
#Create redshift cluster
def create_red_shift():
    redshift = boto3.client('redshift',
                        region_name='us-east-1',
                        aws_access_key_id=os.environ.get("AWS_ACCESS_KEY"), 
                        aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESSKEY"))
    return redshift

redshift = create_red_shift()
clusters = redshift.describe_clusters(ClusterIdentifier='stock-project-cluster')
if len(clusters['Clusters']) > 0:
    print('Cluster already exists')
else:
    response = redshift.create_cluster(
        ClusterIdentifier='stock-project-cluster',
        NodeType=os.environ.get("NODE_TYPE"),
        ClusterType='single-node',
        DBName=os.environ.get("DB_NAME"),
        MasterUsername=os.environ.get("REDSHIFT_USER"),
        MasterUserPassword=os.environ.get("REDSHIFT_PASS"),
    )

Cluster already exists


In [93]:
#Setup connection to redshift and create cursor
conn = setup_connection()
cur = conn.cursor()
#Create tables
create_metadata_table(cur)
create_stock_price_table(cur)
print(cur)

Connected to Redshift
metadata table created
stock_price table created
<cursor object at 0x108c825c0; closed: 0>


In [94]:
%store -r metadata_key
%store -r stockprice_key
print(metadata_key)

V/metadata/2023/03/20230313.csv


In [95]:
# Copy metadata from S3 to Redshift table
copy_metadata_to_redshift = f"""
    COPY metadata
    FROM 's3://stockde/{metadata_key}'
    ACCESS_KEY_ID '{os.environ.get("AWS_ACCESS_KEY")}'
    SECRET_ACCESS_KEY '{os.environ.get("AWS_SECRET_ACCESSKEY")}'
    IGNOREHEADER 1
    CSV
"""

# Copy stockprice from S3 to Redshift table
copy_stockprice_to_redshift = f"""
    COPY stock_price
    FROM 's3://stockde/{stockprice_key}'
    ACCESS_KEY_ID '{os.environ.get("AWS_ACCESS_KEY")}'
    SECRET_ACCESS_KEY '{os.environ.get("AWS_SECRET_ACCESSKEY")}'
    IGNOREHEADER 1
    CSV
"""

In [97]:
cur.execute(copy_metadata_to_redshift)
conn.commit()

cur.execute(copy_stockprice_to_redshift)
conn.commit()


In [92]:
conn.close()
cur.close()